In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn import ensemble

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import learning_curve, validation_curve, StratifiedKFold
import time

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style="ticks")

In [5]:
X = pd.read_csv('C:/Users/kotsi/Desktop/x_train.csv',delimiter=';')
Y = pd.read_csv('C:/Users/kotsi/Desktop/y_train.csv',delimiter=';',header=None)
Y.columns=['Exit']
Y=Y['Exit']
#X=X.drop(['totalBonusScore','totalStarsCount','totalNumOfAttempts'], axis='columns')
X = (X - X.mean()) / X.std()

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 11)
X_train.head()

,maxPlayerLevel,numberOfAttemptedLevels,attemptsOnTheHighestLevel,totalNumOfAttempts,averageNumOfTurnsPerCompletedLevel,doReturnOnLowerLevels,numberOfBoostersUsed,fractionOfUsefullBoosters,totalScore,totalBonusScore,totalStarsCount,numberOfDaysActuallyPlayed
24842,3.300215,1.199518,-0.222800,2.204349,1.205712,2.357930,2.126362,-0.021203,1.181929,1.261199,1.296289,4.077221
21728,-0.417633,-0.108990,-0.370584,-0.396104,0.032296,-0.424084,0.014898,1.156850,-0.113390,-0.052173,-0.030081,-0.623812
10103,-0.120205,0.352836,-0.222800,-0.121056,0.307117,-0.424084,0.165717,0.742350,0.314675,0.397138,0.446052,-0.262194
10529,-0.665489,-0.493846,-0.370584,-0.521126,-0.325028,-0.424084,-0.588378,1.156850,-0.537941,-0.420839,-0.438195,-0.623812
9823,1.267791,-0.262933,-0.370584,0.479049,1.212149,2.357930,0.618173,-0.501150,-0.429021,-0.593651,-0.472204,1.545896


## Рандомный лес

In [6]:
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=11, verbose=0, warm_start=False)

In [7]:
y_train_predict = rf.predict(X_train)
y_test_predict = rf.predict(X_test)

In [8]:
print(accuracy_score(y_test, y_test_predict), precision_score(y_test, y_test_predict), recall_score(y_test, y_test_predict))

0.8103993673388691 0.6740088105726872 0.6406140963014655


In [25]:
parameters = {
    'n_estimators'      : [365], # число деревьев в лесу
    'max_depth'         : [9],
    'max_features': ['log2'],
    'criterion' :['gini']
}
grid = GridSearchCV(ensemble.RandomForestClassifier(), parameters, cv=5) 
grid.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [365, 370, 375], 'max_depth': [9], 'max_features': ['log2'], 'criterion': ['gini']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [26]:
grid.best_estimator_.n_estimators

365

In [20]:
grid.best_estimator_.criterion

'gini'

In [28]:
rf = ensemble.RandomForestClassifier(n_estimators=365, max_depth=9, max_features='log2',criterion='gini', random_state=11)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=9, max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=365, n_jobs=None,
            oob_score=False, random_state=11, verbose=0, warm_start=False)

In [29]:
y_train_predict = rf.predict(X_train)
y_test_predict = rf.predict(X_test)

In [30]:
print(accuracy_score(y_test, y_test_predict), precision_score(y_test, y_test_predict), recall_score(y_test, y_test_predict))

0.8238434163701067 0.7094281298299846 0.6406140963014655


## Бустинг

In [31]:
boost=ensemble.GradientBoostingClassifier()
boost.fit(X_train, y_train)
y_train_predict = boost.predict(X_train)
y_test_predict = boost.predict(X_test)

In [32]:
print(accuracy_score(y_test, y_test_predict), precision_score(y_test, y_test_predict), recall_score(y_test, y_test_predict))

0.8216686437327007 0.7053364269141531 0.636427076064201


In [87]:
parameters = {
    'n_estimators':[14],
    'max_depth':[12],
    'min_samples_split':[850],
 'min_samples_leaf':[58],
    'max_features':[9],
    'subsample':[0.72,0.725,0.73]
}    
grid = GridSearchCV(ensemble.GradientBoostingClassifier(), parameters) 
grid.fit(X_train, y_train)

C:\Users\kotsi\Anaconda37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_sampl...      subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [14], 'max_depth': [12], 'min_samples_split': [850], 'min_samples_leaf': [58], 'max_features': [9], 'subsample': [0.72, 0.725, 0.73]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [88]:
grid.best_estimator_.subsample

0.73

In [93]:
boost=ensemble.GradientBoostingClassifier(n_estimators=14, max_depth=12,min_samples_split=850,
                                          min_samples_leaf=58, max_features=9,subsample=0.73)
boost.fit(X_train, y_train)
y_train_predict = boost.predict(X_train)
y_test_predict = boost.predict(X_test)

In [94]:
print(accuracy_score(y_test, y_test_predict), precision_score(y_test, y_test_predict), recall_score(y_test, y_test_predict))

0.82502965599051 0.7395104895104895 0.5903698534542917


## AdaBoost

In [7]:
ada=ensemble.AdaBoostClassifier()
ada.fit(X_train, y_train)
y_train_predict = ada.predict(X_train)
y_test_predict = ada.predict(X_test)

In [8]:
print(accuracy_score(y_test, y_test_predict), precision_score(y_test, y_test_predict), recall_score(y_test, y_test_predict))

0.8196915776986952 0.6878154289834174 0.6657362177250523


In [9]:
param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "n_estimators": [1, 2]
             }

DTC = DecisionTreeClassifier(random_state = 11, max_features = "auto", class_weight = "auto",max_depth = None)

ABC = ensemble.AdaBoostClassifier(base_estimator = DTC)

In [11]:
params = {'base_estimator': [ensemble.AdaBoostClassifier(), ensemble.GradientBoostingClassifier()]}
gs = GridSearchCV(ensemble.AdaBoostClassifier(base_estimator=DecisionTreeClassifier()), params)
gs.fit(X_train,y_train)

C:\Users\kotsi\Anaconda37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=1.0, n_estimators=50, random_state=None),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'base_estimator': [AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None), GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              m...    subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm

In [12]:
gs.best_params_

{'base_estimator': AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
           learning_rate=1.0, n_estimators=50, random_state=None)}

In [16]:
boost=ensemble.AdaBoostClassifier(base_estimator=ensemble.AdaBoostClassifier())
boost.fit(X_train, y_train)
y_train_predict = boost.predict(X_train)
y_test_predict = boost.predict(X_test)
print(accuracy_score(y_test, y_test_predict), precision_score(y_test, y_test_predict), recall_score(y_test, y_test_predict))

0.8208778173190985 0.6921954777534646 0.6622470341939986
